In [1]:
from ssri_interactions.io import load_derived_generic
from ssri_interactions.transforms import SpikesHandler
from ssri_interactions.surrogates import shuffle_spikes
from ssri_interactions.transforms.brain_state import StateHandler, RawEEGHandler
from ssri_interactions.transforms.brain_state_spikes import (
    align_spikes_to_states_long, align_spikes_to_phase_long, align_bins_to_states_long,
    )
from ssri_interactions.transforms.nbox_transforms import segment_spikes
from ssri_interactions.spiketrains.spiketrain_stats import cv_isi_burst
from ssri_interactions.config import ExperimentInfo, Config
from ssri_interactions.responders.brain_state import SpikeRateResonders, PhaseLockResponders
from ssri_interactions.spiketrains.neurontype_props import ChiSquarePostHoc
from ssri_interactions.plots.circular import circular_hist

from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import numpy as np
import pandas as pd
import warnings

sns.set_theme(context="poster", style="ticks")

from IPython.display import display

/opt/miniconda3/envs/ssri/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Spike Rate Change During EEG States


### Load Data

In [2]:
states_path = Config.derived_data_dir / "lfp_states.csv"
session_names = pd.read_csv(states_path).query("quality == 'good'").session_name.unique().tolist()

neuron_types = load_derived_generic("neuron_types.csv")
states_handler = StateHandler(
    states_path=states_path,
    quality_to_include=("good", "med"),
    t_start=0,
    t_stop=1800,
    session_names=session_names,
)
spikes_handler = SpikesHandler(
    block="pre",
    t_start=0,
    bin_width=1,
    t_stop=1800,
    session_names=session_names,
)

df_aligned = align_bins_to_states_long(
    spikes_handler=spikes_handler,
    states_handler=states_handler,
    neuron_types=neuron_types
)
df_aligned["zcounts"] = (
    df_aligned
    .groupby("neuron_id")["counts"]
    .transform(zscore)
)

### Calculate Responders

- Mixed ANOVA for interactions within neurons (brain states) and among neurons (neuron types)
- Post hoc responder status for each neuron using Mann-Whitney U test 

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mod = SpikeRateResonders(df_value_col="zcounts", round_output=2)
    anova, contrasts = mod.get_anova(df_aligned, fit_neuron_types=True)

    display(anova)
    # display(contrasts)

    responders = mod.get_responders(df_aligned, abs_diff_thresh=0.1)
    display(responders.sample(3))


    responders_raw = (
        SpikeRateResonders(df_value_col="counts", round_output=2)
        .get_responders(df_aligned, abs_diff_thresh=0.1)
        .rename(columns={"Diff": "Diff_raw"})
    )

    (
        responders
        .merge(responders_raw[["neuron_id", "Diff_raw"]], how="left", on="neuron_id")
        .to_csv(
            Config.derived_data_dir / "brain_states_spikerate_responders.csv",
            index=False,
        )
    )

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,neuron_type,0.04,2,470,0.02,2.09,0.12,0.01,NaN
1,state,3.14,1,470,3.14,23.63,0.00,0.05,1.0
2,Interaction,0.23,2,470,0.11,0.85,0.43,0.00,NaN


,neuron_id,n_sw,n_act,Mean_sw,Mean_act,Diff,U,p,sig
40,1220,1281.0,518.0,-0.34,0.85,1.19,119527.5,0.00,True
53,1235,1281.0,518.0,-0.06,0.14,0.20,326941.0,0.66,False
183,1393,1088.0,711.0,-0.06,0.09,0.14,355951.0,0.00,True


# Phase Locking Analysis

### Load Data and Align to EEG Oscillation Phase
- Raw EEG signal downsampled to 250 Hz
- In activated brain states, it is filtered between 4 - 8 Hz
- In slow wave states, it is filtered between 0.5 - 4 Hz
- Spike times are aligned to EEG phase separately for each brain state
- In each state, the distrobution of phases is tested for uniformity using Rayleigh tests
- The prefered phase of each neuron and whether it is significantly different phase locked is saved in a file
- This file is loaded into R and analysed using an GLM on angular embeddings (see below)

In [4]:
from ssri_interactions.io import load_lfp_raw

spikes_handler = SpikesHandler(
    block="pre",
    t_start=0,
    bin_width=1,
    t_stop=1800,
    session_names=session_names,
)

eeg_handler = RawEEGHandler(
    block="pre",
    t_start=0,
    t_stop=1800,
    session_names=session_names,
    loader=load_lfp_raw
)
df_aligned_phase = align_spikes_to_phase_long(
    spikes_handler=spikes_handler,
    states_handler=states_handler,
    raw_eeg_handler=eeg_handler,
    neuron_types=None,
).dropna()


df_sw = df_aligned_phase.query("state == 'sw'")
df_act = df_aligned_phase.query("state == 'act'")

mod = PhaseLockResponders(round_output=2, fs=(250 * 6) / (2 * np.pi))
df_res_act = mod.prefered_angles(df_act, phase_col="theta_phase")
df_res_sw = mod.prefered_angles(df_sw, phase_col="delta_phase",)
df_prefered_angles = pd.concat([(
        df_res_sw
        .assign(oscillation="delta")
        [["neuron_id", "oscillation", "mean_angle", "var", "p"]]
        ),
        (
            df_res_act
            .assign(oscillation="theta")
            [["neuron_id", "oscillation", "mean_angle", "var", "p"]]
        )
]
)
df_prefered_angles = df_prefered_angles.merge(neuron_types)
display(df_prefered_angles.sample(3))
df_prefered_angles.to_csv(Config.derived_data_dir / "brain_states_phase_responders.csv", index=False)

,neuron_id,oscillation,mean_angle,var,p,session_name,group_name,experiment_name,group,neuron_type,width_basepost,mean_firing_rate,cv_isi_burst
131,1251,theta,2.65,0.96,0.56,hamilton_23,citalopram_continuation,HAMILTON,CIT,SIR,NaN,0.506024,1.005765
695,2062,delta,0.47,0.92,0.00,hamilton_26,citalopram_discontinuation,HAMILTON,DIS,SIR,31.561830,4.811259,0.666311
464,1463,theta,2.26,0.93,0.28,hamilton_08,citalopram_continuation,HAMILTON,CIT,SIR,37.890437,1.398162,0.875790


In [16]:
df_prefered_angles = df_prefered_angles.assign(sig=lambda x: x.p < 0.05)
mod = ChiSquarePostHoc(value_col="sig", round=2)

display(mod(df_prefered_angles.query("oscillation == 'delta' and group == 'CIT'")))
display(mod(df_prefered_angles.query("oscillation == 'delta' and group == 'SAL'")))

display(mod(df_prefered_angles.query("oscillation == 'theta' and group == 'CIT'")))
display(mod(df_prefered_angles.query("oscillation == 'theta' and group == 'SAL'")))

anova                          Chi2(2)=2.2 (p=0.33)
SR - SIR    52.63%; 55.91% | Chi(1.0)=0.09 (p=0.76)
SR - FF     52.63%; 83.33% | Chi(1.0)=1.09 (p=0.56)
SIR - FF    55.91%; 83.33% | Chi(1.0)=0.79 (p=0.56)
dtype: object

anova                         Chi2(2)=6.5 (p=0.04*)
SIR - FF      89.29%; 90.91% | Chi(1.0)=0.0 (p=1.0)
SIR - SR    89.29%; 66.67% | Chi(1.0)=3.69 (p=0.16)
FF - SR     90.91%; 66.67% | Chi(1.0)=1.52 (p=0.33)
dtype: object

anova                         Chi2(2)=7.2 (p=0.03*)
SR - SIR    38.95%; 24.73% | Chi(1.0)=3.74 (p=0.16)
SR - FF        38.95%; 0.0% | Chi(1.0)=2.2 (p=0.21)
SIR - FF      24.73%; 0.0% | Chi(1.0)=0.79 (p=0.37)
dtype: object

anova                          Chi2(2)=3.2 (p=0.20)
SIR - FF    46.43%; 72.73% | Chi(1.0)=1.27 (p=0.39)
SIR - SR    46.43%; 64.44% | Chi(1.0)=1.61 (p=0.39)
FF - SR     72.73%; 64.44% | Chi(1.0)=0.03 (p=0.87)
dtype: object